In [39]:
from requests import get
from bs4 import BeautifulSoup
from IPython.core.display import clear_output
from time import sleep, time
from random import randint
import warnings
import matplotlib.pyplot as plt
import pandas as pd
import string

In [ ]:
##These labels are used to create the webpage of the yelp restaurants

In [16]:
website = "https://www.yelp.com/search?find_desc=Restaurants"
sf_label = "&find_loc=San%20Francisco%2C%20CA"
sj_label = "&find_loc=San%20Jose%2C%20CA"
cheap_label = "&attrs=RestaurantsPriceRange2.1"
medium_label = "&attrs=RestaurantsPriceRange2.2"
expensive_label = "&attrs=RestaurantsPriceRange2.3"
vegetarian_label ="&cflt=vegetarian"

In [ ]:
## The format is:
##(Website + Location_label + Price_label + page_Label) <nonveg
##(Website + location_label + price_label + vegetarian+label) < veg
## We cannot use a loop to scrape all the categories at once because certain categories have many more restaurants than others
## therefore, you have to just run this code procedurally for each tag.
## also, some pages have random advertisement restaurants that we have to get rid of as well
## If we insist on using a loop we have to decided exactly what locations/how many of each restaurant we are using

In [18]:

def getPage(inp):
    return ((inp-1)*30)

In [19]:
##insert page number as an arg. here
page = getPage(1)
page_label = "&start=" + str(page)

In [35]:
##making as a list so we dont have to refactor as much in case we decide to change back to using a list 


newSite = get(website+sf_label+cheap_label+page_label)
toScrape.append(newSite)
html_soup = BeautifulSoup(newSite.text, 'html.parser')

In [37]:
restaurant_containers = html_soup.find_all('div', class_ = 'lemon--div__373c0__1mboc searchResult__373c0__1yggB border-color--default__373c0__2oFDT')
refList = []
for current in restaurant_containers:
    title = current.h3.find('a', class_ = "lemon--a__373c0__IEZFH link__373c0__29943 link-color--blue-dark__373c0__1mhJo link-size--inherit__373c0__2JXk5")
    refList.append(title['href'])
    

    


['/adredir?ad_business_id=Qe1c3ePgO9TkGSRsgRdTCw&campaign_id=vb_Xcd_ujY_5LuIIEMPD0g&click_origin=search_results&placement=above_search&redirect_url=https%3A%2F%2Fwww.yelp.com%2Fbiz%2Fmcdonalds-san-francisco-28&request_id=e89c0431cff08625&signature=777a29e664833b5fadf8f398daa6927319772aeb5e5e9bccbbaf7698daf94d1c&slot=0', '/biz/the-bird-san-francisco?osq=Restaurants', '/biz/tselogs-san-francisco-5?osq=Restaurants', '/biz/nicks-crispy-tacos-san-francisco-2?osq=Restaurants', '/biz/taboun-san-francisco?osq=Restaurants', '/biz/dragoneats-san-francisco-7?osq=Restaurants', '/biz/yama-chan-san-francisco?osq=Restaurants', '/biz/la-taqueria-san-francisco-2?osq=Restaurants', '/biz/el-farolito-san-francisco-2?osq=Restaurants', '/biz/pushkin-san-francisco?osq=Restaurants', '/biz/dinosaurs-san-francisco-4?osq=Restaurants', '/biz/palmyra-san-francisco?osq=Restaurants', '/biz/house-of-pancakes-san-francisco?osq=Restaurants', '/biz/super-duper-burgers-san-francisco-4?osq=Restaurants', '/biz/truly-medite

In [41]:
yelp_template = string.Template("https://www.yelp.com$link")
for i in range(len(refList)):
    refList[i] = yelp_template.substitute(link = refList[i])
    
    
#removing the first restaurant bc its an advertised restaurant I think
refList.pop(0)
refList



['https://www.yelp.comhttps://www.yelp.com/adredir?ad_business_id=Qe1c3ePgO9TkGSRsgRdTCw&campaign_id=vb_Xcd_ujY_5LuIIEMPD0g&click_origin=search_results&placement=above_search&redirect_url=https%3A%2F%2Fwww.yelp.com%2Fbiz%2Fmcdonalds-san-francisco-28&request_id=e89c0431cff08625&signature=777a29e664833b5fadf8f398daa6927319772aeb5e5e9bccbbaf7698daf94d1c&slot=0',
 'https://www.yelp.comhttps://www.yelp.com/biz/the-bird-san-francisco?osq=Restaurants',
 'https://www.yelp.comhttps://www.yelp.com/biz/tselogs-san-francisco-5?osq=Restaurants',
 'https://www.yelp.comhttps://www.yelp.com/biz/nicks-crispy-tacos-san-francisco-2?osq=Restaurants',
 'https://www.yelp.comhttps://www.yelp.com/biz/taboun-san-francisco?osq=Restaurants',
 'https://www.yelp.comhttps://www.yelp.com/biz/dragoneats-san-francisco-7?osq=Restaurants',
 'https://www.yelp.comhttps://www.yelp.com/biz/yama-chan-san-francisco?osq=Restaurants',
 'https://www.yelp.comhttps://www.yelp.com/biz/la-taqueria-san-francisco-2?osq=Restaurants',
 